# WEEK2      
> ## 经典网络      
### LeNet-5      
### AlexNet

# WEEK3     
> ## 目标定位  

图片首先通过一个CNN，输出一个softmax分辨图内部有什么内容(车、标志、障碍物、背景)       
如果需要定位，那么要在softmax当中输出一个bounding box($b_x,b_y,b_h,b_w$)      
以图片左上角为坐标轴零点(0,0)
$b_x$:中心点x坐标     
$b_y$:中心点y坐标     
$b_h$:bounding box宽度    
$b_w$:bounding box长度      
<img style="float: center;" src="course_4_pics/week_3_1.PNG" width=200 height=200>      

### 如何定义$y$    
```
例:
四个分类:车、标志、障碍物、背景
```
定义y为:    
$$y=\left[
\begin{matrix}
P_c\\
b_x\\
b_y\\
b_w\\
b_h\\
c_1\\
c_2\\
c_3\\
\end{matrix}
\right]
$$        
$P_c$:图片中是否有物体，如果有设置为1；如果没有设置为0，其余参数设置为不重要     
$b_x,b_y$:中心点x,y坐标     
$b_w,b_h$:bounding box宽度与长度   
$P_c$:如果有物体，三个物体的类别    

损失函数定义如下:    
$$L(\hat y ,y) = \begin{cases}
\sum_i^8(\hat y_i-y_i)^2,&if\ y_1=1\\
(\hat y_1-y_1)^2,&if\ y_1=0
\end{cases}$$          

通常情况下，$P_c$可以使用逻辑回归，bounding box参数使用平方误差，$P_c$类别使用softmax     

> ## 特征检测    

标注出人脸、人的身体上的特征点(眼角，嘴角等)，组成x，y坐标对，通过卷积神经网络识别。    

> ## 目标检测     

### 基于滑动窗口的目标检测    

1. 选择一批剪裁至合理大小的数据集(车辆、无车辆的背景等)，输出一个convnet输出label——1/0    
<img style="float: center;" src="course_4_pics/week_3_2.PNG" width=400 height=400>      

2. 选定图片一个特定大小的窗口输入convnet输出label，以一个步幅移动窗口直至图片结束。之后选择更大的窗口进行扫描。    
<img style="float: center;" src="course_4_pics/week_3_3.PNG" width=400 height=400>          

小的步幅会导致计算成本增高，大步幅会导致颗粒度过大因而影响卷积神经网络的性能。   

### 卷积滑动窗口的目标检测(优化)     

**1. 全连接层转化为卷积层**    
第一个CONV和FC连接的权重视为用400个$5\times 5\times 16$的卷积核进行连接       
第二个用400个$1\times 1\times 400$的卷积核进行连接   
第三个用4个$1\times 1\times 400$的卷积核进行连接

<img style="float: center;" src="course_4_pics/week_3_4.PNG" width=600 height=600>           

**2. 使用卷积神经网络一次性运算滑窗**        
如图，第一个神经网络是已经**训练完成**的车辆分类器      
第二个神经网络将较大的需要滑窗的图片输入，第一个卷积核根据步幅扫描图片，最终输出的是四个像素——每个像素代表 红、黄、紫、绿 对应的滑动窗口，**可以在卷积中共享计算——因为训练完成所以卷积核的参数不变，相交部分用同一个卷积核扫描同一个图像**，并且不用限制输入图像的大小。
<img style="float: center;" src="course_4_pics/week_3_5.PNG" width=600 height=600>      

> ## Bounding Box预测    

YOLO算法(You only look once):    
1. 先将图片分成多个cell，训练集中定义每个cell的label       
$$y=\left[
\begin{matrix}
P_c\\
b_x\\
b_y\\
b_w\\
b_h\\
c_1\\
c_2\\
c_3\\
\end{matrix}
\right]
$$
对于图中的图片，最终输出的是一个$3\times 3\times 8$的结果
<img style="float: center;" src="course_4_pics/week_3_6.PNG" width=300 height=300>      

2. 
对于图中的第一个cell而言，$P_c=0$，因此其他label是不重要的。     
对于图中的第四个和第六个cell，已知图中有两个对象，分别取两个对象的中心点，将对象分配到中心点所在的cell当中(第四个和第六个)      

<img style="float: center;" src="course_4_pics/week_3_7.PNG" width=600 height=600>

3. 输入convnet得出九个格子当中有object的cell，并输出这个cell的边界        

**label数值的设置**：
通常情况下选择相对于cell的数值，设一个cell的左上角为(0,0)，右下角为(1,1)     
那么bounding box的长宽可能大于1，中心点的坐标一定小于1
<img style="float: center;" src="course_4_pics/week_3_8.PNG" width=600 height=600>          

> ## 交并比(intersection over Union,IoU)     

算法给出一个边框，实际人工标注了一个边框     
交并比函数做的是，计算两个边框的**交集与并集的比例**        
通常认为算法正确当**IoU>0.5**        
<img style="float: center;" src="course_4_pics/week_3_9.PNG" width=600 height=600>      

> # 非极大值抑制     

### No max suppression:        

<img style="float: center;" src="course_4_pics/week_3_10.PNG" width=600 height=600>
1. 考虑一个格子只输出一个object，每个object的预测结果是      
$$y=\left[
\begin{matrix}
P_c\\
b_x\\
b_y\\
b_w\\
b_h\\
\end{matrix}
\right]
$$      
去掉$P_c\leq 0.6$的检测点      


2. 选择$P_c$最高的点，计算其余输出的框与最高的框的交并比，删除交并比$IoU>0.5$的检测点，输出该最高的检测点    

3. 选择下一个$P_c$最高的点，重复直至所有点都输出完成     

<img style="float: center;" src="course_4_pics/week_3_11.PNG" width=600 height=600>     

> ## Anchor Box    

当你需要在一个格子当中检测出多个对象——anchor box    
定义两个anchorbox，为每个anchorbox有个label的定义     
训练图像中的每个对象都分配给网格单元格，网格单元格包含具有最高IOU的网格单元的对象小点和锚框。
<img style="float: center;" src="course_4_pics/week_3_12.PNG" width=600 height=600>
> ## YOLO算法   

> ## 候选区域(Faster R-CNN)

# WEEK4    

> ## 人脸识别   

**Face Verification人脸验证问题**(1:1问题)
- 输入图像、名称/ID
- 输出输入图像是否为身份对应的人

**Face Recognition人脸识别问题**(1:K问题)
- 有一个K人的数据库
- 获取输入图像
- 如果图像是K人中的任何一个，则输出ID(或“无法识别”)

假设使用人脸验证系统，对应K个人的ID，在1:1问题当中的失败概率是1%。那么在1:K当中失败的概率由于要识别K个人，失败概率大大提升($1-0.99^K$)

> ## One-shot学习     

对一个人的一个样例进行学习来确认一个人的身份(此时因为训练样本小，K个人只有K个样本；并且加入新员工需要重新训练，使用softmax效果并不好)      
### 学习"similarity" function      
一个函数$d$输出两张图片的相似度    
$d(img1,img2)=degree\ of\ difference\ between\ images$    
$if\ d(imag1,imag2)>\tau$，那么认为两个人是同一个人    
$if\ d(imag1,imag2)\leq \tau$，那么认为两个人不是同一个人      

> ## Siamese network     

输入一个$x^{(1)}$经过一个卷积神经网络，在最后的全连接层(128维）输出一个该图片的编码$f(x^{(1)})$，定义$d(x^{(1)},x^{(2)})=||f(x^{(1)})-f(x^{(2)})||^2$，**通过学习参数使得函数$d$在两个人是同一个人时取得较小的值**   

<img style="float: center;" src="course_4_pics/week_4_1.PNG" width=600 height=600> 

<img style="float: center;" src="course_4_pics/week_4_2.PNG" width=600 height=600> 

> ## triplet function

定义三元组损失方程(A:anchor;P:positive;N:negtive)      
首先由两组图片，一组由锚图A与正例P构成，一组由锚图A与反例N构成   
我们想要的是$d(A,P)\leq d(A,N)$，即$||f(A)-f(P)||^2-||f(A)-f(N)||\leq 0$,通常定义一个$\alpha$在左边，叫做margin(需要margin的原因是当图片编码输出全部为0时式子也成立)，即得到$||f(A)-f(P)||^2-||f(A)-f(N)||+\alpha \leq 0$
  
<img style="float: center;" src="course_4_pics/week_4_3.PNG" width=600 height=600> 

### Loss Function
**损失函数:**
损失函数基于三个图片——$A,P,N$     
$$Loss(A,P,N)=max(||f(A)-f(P)||^2+(\alpha)-||f(A)-f(N)||^2,0)$$
损失函数中**距离差只要小于0(或$\alpha$)**，那么得到一个0，这样可以保证损失函数是一个正数     

**代价函数:**
$$J=\sum_i^ML(A^{(i)},P^{(i)},N^{(i)})$$

**训练集:**    
1. 在一定数量的图片中选择三元组(同一个人要有多张照片)     
2. 选择训练集三元组时如果随机选择，那么$d(A,P)\leq d(A,N)$很容易满足。所以需要选择难例训练，梯度下降容易起效。     

### 人脸识别与二分类     
将人脸识别转化为二分类问题   
将两张图片输入两个CNN当中，最后得到两串编码，两串编码输入一个逻辑斯蒂回归进行二分类。   
$$\hat y=\sigma(\sum_k^{128}w_k|f(x^{(i)})_k-f(x^{(j)})_k|+b)$$     
也可以采用其他的计算编码间距离的方式:
$$\hat y=\sigma(\sum_k^{128}w_k\frac{(f(x^{(i)})_k-f(x^{(j)})_k)^2}{f(x^{(i)})_k+f(x^{(j)})_k}+b)$$
<img style="float: center;" src="course_4_pics/week_4_4.PNG" width=600 height=600> 
 
 <img style="float: center;" src="course_4_pics/week_4_5.PNG" width=600 height=600>

> ## 神经风格转换   

通过卷积神经网络转换图片的风格(给定一个原始图片，一个风格图片，改变原始图片的风格)   
<img style="float: center;" src="course_4_pics/week_4_6.PNG" width=300 height=300> 
### 深度卷积神经网络      

找到最大化激活各个卷积层的图片——浅层倾向于一些简单的特征图，例如边缘检测等；深层倾向于复杂的图片，例如五官、车轮、动物以及人脸等。
<img style="float: center;" src="course_4_pics/week_4_7.PNG" width=600 height=600> 

### 代价函数    
为了神经风格转化，定义一个关于生成图像$G$的代价函数:     
$$J(G)=\alpha J_{content}(C,G)+\beta J_{style}(S,G)$$    
代价函数包含**内容代价函数与风格代价函数**，分别表示**生成图片$G$与内容图片$C$的相似度与生成图片$G$与风格图片$S$的相似度**，并用超参数权重$\alpha$与$\beta$权衡两者     

**图片生成步骤:**     
1.随机生成图片**G**     
2.使用梯度下降更新生成图片**G**       
$$G=G-\frac{\partial J(G)}{\partial G}$$      

### 内容代价函数      

- 使用隐藏层$l$层计算内容代价函数    
- 使用一个预训练过的卷积神经网络(VGG等)   
- $a^{[l](C)}$和$a^{[l](G)}$为图片C与G分别在$l$层的激活值。     
- 如果$a^{[l](C)}$和$a^{[l](G)}$很接近，那么代表他们具有相似的内容。——$l$通常选择不那么深也不那么浅的层        

定义内容代价函数:    

$$J_{content}(C,G) = \frac{1}{2}||a^{[l](C)}-a^{[l](G)}||2$$


### 风格代价函数     

